In [1]:
import shap
import torch
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
import numpy as np
import pandas as pd

In [ ]:

# Load RoBERTa Sentiment Analysis Model
roberta_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name)
roberta_model.eval()

def analyze_roberta_sentiment(text, tokenizer, model, threshold=0.5):
    """
    Analyze sentiment using RoBERTa and classify as Positive, or Negative.
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = roberta_model(**inputs)
        logits = outputs.logits

    probs = softmax(logits, dim=-1).numpy()[0]  # Convert logits to probabilities
    negative_prob, neutral_prob, positive_prob = probs  # Extract individual probabilities

    # Classify based on confidence threshold
    if positive_prob >= threshold:
        return "Positive"
    else:
        return "Negative"

# Load Data
df = pd.read_csv("new_hannel_partner_feedback.csv")  # Ensure it has 'text' and 'true_label' columns

# Analyze Sentiment for RoBERTa
df["roberta_model_prediction"] = df["Feedback_Text"].apply(lambda x: analyze_roberta_sentiment(x, tokenizer, roberta_model))

# Print classification report
print("\n RoBERTa Model Performance:")
print(classification_report(df["Sentiment"], df["roberta_model_prediction"]))


FileNotFoundError: [Errno 2] No such file or directory: 'channel_partner_feedback.csv'

In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
import shap

# Load RoBERTa model and tokenizer
roberta_model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(roberta_model_name)
roberta_model = AutoModelForSequenceClassification.from_pretrained(roberta_model_name)
roberta_model.eval()

# Load data
df = pd.read_csv("channel_partner_feedback.csv")

# Binary prediction function for SHAP
def binary_sentiment_predict(texts):
    all_probs = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
        with torch.no_grad():
            outputs = roberta_model(**inputs)
            logits = outputs.logits
        probs = softmax(logits, dim=-1).numpy()[0]
        # Extract only Negative (index 0) and Positive (index 2)
        binary_probs = [probs[0], probs[2]]
        all_probs.append(binary_probs)
    return all_probs

# Create SHAP explainer
explainer = shap.Explainer(binary_sentiment_predict, tokenizer)

# Sample a few rows to explain
sample_texts = df["Feedback_Text"].dropna().sample(3, random_state=42).tolist()

# Compute SHAP values
shap_values = explainer(sample_texts)

# Plot explanation for the first sample
shap.plots.text(shap_values[0])


  0%|          | 0/110 [00:00<?, ?it/s]


PartitionExplainer explainer:  33%|███▎      | 1/3 [00:00<?, ?it/s]

  0%|          | 0/156 [00:00<?, ?it/s]


PartitionExplainer explainer: 100%|██████████| 3/3 [01:04<00:00, 12.50s/it]

  0%|          | 0/182 [00:00<?, ?it/s]

PartitionExplainer explainer: 4it [01:37, 32.46s/it]
